In [49]:
use RIVA_Azure_Data_Analytics_Project_D3M
go

with cte_emp_levels as (
	select EmployeeId, RivaLevelUtil, PositionName, PositionCode, BusinessUnit
	from (
		select
			case
				when e.[Position Code] like '% VII%' then 0.25
				when e.[Position Code] like '% VI%' then 0.5
				when e.[Position Code] like '% IV%' then 0.83
				when e.[Position Code] like '% V%' then 0.75
				when e.[Position Code] like '% III%' then 0.85
				when e.[Position Code] like '% II%' then 0.93
				when e.[Position Code] like '% I%' then 0.95
				else 1.0
			end as RivaLevelUtil
			, value as PositionName
			, e.[Employee Id] as EmployeeId
			, e.[Position Code] as PositionCode
			, case 
				when e.[Position Code] like 'CORP%' or e.[Position Code] = 'X' then 'CORP'
				when e.[Position Code] like 'CIV%' then 'FEDCIV'
				when e.[Position Code] like 'DOD%' then 'DOD/NS'
				when e.[Position Code] like 'HEA%' then 'FEDHEALTH'
			end as BusinessUnit
		from 
			deltek.EmployeeList as e
			cross apply string_split(e.[Position Code], ' ')
	) as t
	group by EmployeeId, RivaLevelUtil, PositionName, PositionCode, BusinessUnit
)

, cte_emp_position_levels as (
	select EmployeeId, PositionCode, PositionName as PositionLevel, RivaLevelUtil, BusinessUnit
	from cte_emp_levels as t 
	where t.PositionName in ('I', 'II', 'III', 'IV', 'V', 'VI', 'VII')
	group by EmployeeId, PositionCode, PositionName, RivaLevelUtil, BusinessUnit
)
, cte_emp as (
    select
        e.[Employee Id] as EmployeeId
        , e.[Position Code] as PositionName
        , concat(e.[Preferred/First Name], ' ', e.[Last Name]) as EmployeeName
        , e.[Employee Type] as EmployeeType
        , e.[Cost Cent Description] as CostCentDescription
        , e.[Current Work Email] as CurrentWorkEmail
        , e.[Hire Date] as HireDate
        , e.[Termination Date] as TerminationDate
        , (select top 1 RivaLevelUtil from cte_emp_position_levels where EmployeeId = e.[Employee Id]) as RivaLevelUtil
        , (select top 1 PositionLevel from cte_emp_position_levels where EmployeeId = e.[Employee Id]) as PositionLevel
        , (select top 1 BusinessUnit from cte_emp_position_levels where EmployeeId = e.[Employee Id]) as BusinessUnit
    from 
        deltek.EmployeeList as e
)

select 
    SUBSTRING(u.[Project ID], 1, 6) as ProjectNameCode
    , u.[Employee ID]
    , u.[Year]
    , u.[Period Number]
    , u.[Status]
    , u.[Project ID]
    , u.[Pay Type]
    , u.[Hours Date]
    , u.[Entered Hours]
    -- Projects
    , p.[Project Name]
    , p.[Project Type Desc]
    , p.[Project Classification]
    , p.[Project Start Date]
    , p.[Project End Date]
    , p.[Total Funded]
    , p.[Cost Funded]
    , p.[Fee Funded]
    , p.[Project Value Cost]
    , p.[Project Value Fee]
    , parent.[Project ID] as [Project ID (6-Digit)]
    , parent.[Project Name] as [Project Name (6-Digit)]
    , parent.[Project Type Desc] as [Project Type Desc (6-Digit)]
    , parent.[Project Classification] as [Project Classification (6-Digit)]
    , parent.[Project Start Date] as [Project Start Date (6-Digit)]
    , parent.[Project End Date] as [Project End Date (6-Digit)]
    , parent.[Total Funded] as [Total Funded (6-Digit)]
    , parent.[Cost Funded] as [Cost Funded (6-Digit)]
    , parent.[Fee Funded] as [Fee Funded (6-Digit)]
    , parent.[Project Value Cost] as [Project Value Cost (6-Digit)]
    , parent.[Project Value Fee] as [Project Value Fee (6-Digit)]
    -- Employees
    , case
        when e.[Position Code] like 'CORP%' or e.[Position Code] = 'X' then 'CORP'
        when e.[Position Code] like 'CIV%' then 'FEDCIV'
        when e.[Position Code] like 'DOD%' then 'DOD/NS'
        when e.[Position Code] like 'HEA%' then 'FEDHEALTH'
        else e.[Position Code]
    end as BU
    , e.[Last Name]
    , e.[Preferred/First Name]
    , e.[Cost Cent Description]
    , e.[Position Code]
    , e.[Hire Date]
    , e.[Current Work Email]
    , e.[Employee Type]
    , e.[Termination Date]
from deltek.Utilization as u

left join deltek.ProjectList as p 
    on u.[Project ID] = p.[Project ID]

left join deltek.ProjectList as parent 
    on p.[Project ID] = SUBSTRING(u.[Project ID], 1, 6)

left join deltek.EmployeeList as e
    on u.[Employee ID] = concat('E00', e.[Employee Id])

Commands completed successfully.

(10 rows affected)

Total execution time: 00:00:00.157

ProjectNameCode Employee ID Year Period Number Status Project ID Pay Type Hours Date Entered Hours Project Name Project Type Desc Project Classification Project Start Date Project End Date Total Funded Cost Funded Fee Funded Project Value Cost Project Value Fee Project ID (6-Digit) Project Name (6-Digit) Project Type Desc (6-Digit) Project Classification (6-Digit) Project Start Date (6-Digit) Project End Date (6-Digit) Total Funded (6-Digit) Cost Funded (6-Digit) Fee Funded (6-Digit) Project Value Cost (6-Digit) Project Value Fee (6-Digit) HHS001 E00741 2021 003 Processed HHS001.0003.01 R 2021-02-02 00:00:00.000 8 SAMHSA Peer Review OY1 COST DIRECT PROJECT 2020-08-30 2021-08-29 695025 695025 0 695025 0 NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL HHS001 E00741 2021 003 Processed HHS001.0003.01 R 2021-02-03 00:00:00.000 8 SAMHSA Peer Review OY1 COST DIRECT PROJECT 2020-08-30 2021-08-29 695025 695025 0 695025 0 NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL HHS001 E00741 2021 003 Processed HHS001.0003.01 R 2021-02-04 00:00:00.000 8 SAMHSA Peer Review OY1 COST DIRECT PROJECT 2020-08-30 2021-08-29 695025 695025 0 695025 0 NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL HHS001 E00741 2021 003 Processed HHS001.0003.01 R 2021-02-05 00:00:00.000 8 SAMHSA Peer Review OY1 COST DIRECT PROJECT 2020-08-30 2021-08-29 695025 695025 0 695025 0 NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL HHS001 E00741 2021 003 Processed HHS001.0003.01 R 2021-02-08 00:00:00.000 8 SAMHSA Peer Review OY1 COST DIRECT PROJECT 2020-08-30 2021-08-29 695025 695025 0 695025 0 NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL HHS001 E00741 2021 003 Processed HHS001.0003.01 R 2021-02-09 00:00:00.000 8 SAMHSA Peer Review OY1 COST DIRECT PROJECT 2020-08-30 2021-08-29 695025 695025 0 695025 0 NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL HHS001 E00741 2021 003 Processed HHS001.0003.01 R 2021-02-10 00:00:00.000 8 SAMHSA Peer Review OY1 COST DIRECT PROJECT 2020-08-30 2021-08-29 695025 695025 0 695025 0 NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL HHS001 E00741 2021 003 Processed HHS001.0003.01 R 2021-02-11 00:00:00.000 8 SAMHSA Peer Review OY1 COST DIRECT PROJECT 2020-08-30 2021-08-29 695025 695025 0 695025 0 NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL HHS001 E00741 2021 003 Processed HHS001.0003.01 R 2021-02-12 00:00:00.000 8 SAMHSA Peer Review OY1 COST DIRECT PROJECT 2020-08-30 2021-08-29 695025 695025 0 695025 0 NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL FRINGE E00741 2021 003 Processed FRINGE.REG.HOL HOL 2021-02-15 00:00:00.000 8 Fringe Regular Holiday INDIRECT INDIRECT NULL NULL 0 0 0 0 0 NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL

In [50]:
select top 100 * from deltek.EmployeeList

(100 rows affected)

Total execution time: 00:00:00.072

Employee Id Last Name Preferred/First Name Cost Cent Description Position Code Current Work Email Hire Date Employee Type Termination Date 100 Abbott Ronald DOJ-BOP MAIL CLERK rabbott@bop.gov 2015-04-29 00:00:00.000 NULL 2017-08-31 00:00:00.000 1000 Norris Adam CORPORATE CORP VI anorris@rivasolutionsinc.com 2021-01-07 00:00:00.000 Non SCA 2022-09-26 00:00:00.000 1001 Hamlin Nicole CORPORATE CORP II nhamlin@rivasolutionsinc.com 2021-01-04 00:00:00.000 Non SCA NULL 1002 Norbury Embrey Jenn Corporate:G&A:Bid&Proposal CORP IV jembrey@rivasolutionsinc.com 2021-01-11 00:00:00.000 Non SCA 2022-09-02 00:00:00.000 1003 Miller Tom CORPORATE VP tmiller@rivasolutionsinc.com 2021-01-11 00:00:00.000 Non SCA 2021-05-21 00:00:00.000 1004 Buker Jacob SBIRS_Air Force NULL jbuker@rivasolutionsinc.com 2021-01-04 00:00:00.000 Union (CO) 2021-06-30 00:00:00.000 1005 Shorter Essie USPTO-OHR-STAFFING CIVFUNCT II eshorter@rivasolutionsinc.com 2021-01-04 00:00:00.000 Non SCA 2022-03-15 00:00:00.000 1006 Noble Clayton NGB-JACO DODNS FUNCT II cnoble@rivasolutionsinc.com 2021-01-04 00:00:00.000 Non SCA 2022-12-01 00:00:00.000 1007 Anthony Joe CORPORATE NULL janthony@rivasolutionsinc.com 2021-01-04 00:00:00.000 NULL 2021-01-05 00:00:00.000 1008 Earles Johnstone Larissa NGB-JACO NULL NULL 2021-01-19 00:00:00.000 Non SCA 2021-06-07 00:00:00.000 1009 Machado Rani USDA Drupal BPA CIVTECH III rmachado@rivasolutionsinc.com 2021-01-12 00:00:00.000 Non SCA NULL 101 Allen Sean DOJ-BOP SUPR PHOTO OP sallen@bop.gov 2010-09-01 00:00:00.000 NULL 2017-08-31 00:00:00.000 1010 Moore Carolyn USPTO-OHR-ADMIN SCACIV cmoore@rivasolutionsinc.com 2021-01-21 00:00:00.000 SCA NULL 1011 Saravia Karla Corporate:G&A:Marketing CORP III ksaravia@rivasolutionsinc.com 2021-01-19 00:00:00.000 Non SCA NULL 1012 Ngo Sophie CORPORATE CORP III sngo@rivasolutionsinc.com 2021-02-08 00:00:00.000 Non SCA 2022-04-06 00:00:00.000 1013 Magana Lisette ARMY IMCOM SCADODNS lmagana@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA NULL 1014 Tayaba Art ARMY IMCOM SCADODNS atayaba@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA NULL 1015 Allard Robert ARMY IMCOM SCADODNS rallard@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA NULL 1016 Franklin DeShauna ARMY IMCOM SCADODNS dfranklin@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA NULL 1018 Falls Brian SBIRS_Air Force NULL NULL 2021-01-20 00:00:00.000 Union (CO) 2021-07-06 00:00:00.000 1019 Arculeo Tony ARMY IMCOM SCADODNS aarculeo@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA NULL 102 Bryant Tanya DOJ-BOP SECRETARY V txbryant@bop.gov 2010-10-01 00:00:00.000 NULL 2017-08-31 00:00:00.000 1020 Arrowood Jamie ARMY IMCOM SCADODNS jarrowood@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA NULL 1021 Barrera Gilbert ARMY IMCOM SCADODNS gbarrera@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA NULL 1022 Howard Kareem ARMY IMCOM SCADODNS khoward@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA 2022-06-30 00:00:00.000 1023 Johnson Jihad ARMY IMCOM SCADODNS jjohnson3@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA NULL 1024 Lamb Krissi ARMY IMCOM SCADODNS klamb@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA NULL 1025 Lopez Christina ARMY IMCOM SCADODNS clopez@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA 2022-10-14 00:00:00.000 1026 Masters Vincent ARMY IMCOM SCADODNS vmasters@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA NULL 1027 Molina Karen ARMY IMCOM SCADODNS kmolina@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA NULL 1028 Sampson Elizabeth or Liz ARMY IMCOM SCADODNS esampson@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA NULL 1029 Vidaurri Ron ARMY IMCOM SCADODNS rvidaurri@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA NULL 103 Cash Javona DOJ-BOP DUP PHOTO OP jcash@bop.gov 2010-09-01 00:00:00.000 NULL 2017-08-31 00:00:00.000 1030 Wenzel Charlie ARMY IMCOM SCADODNS cwenzel@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA NULL 1031 Wright Jenna ARMY IMCOM SCADODNS jwright@rivasolutionsinc.com 2021-02-01 00:00:00.000 SCA 2022-10-07 00:00:00.0

In [44]:
with cte_project_id_parts as (
    select
        ProjectId
        , Value as Part
        , row_number() over(partition by ProjectId order by ProjectId) as PartNum
    from (
            select [Project ID] as ProjectId 
            from deltek.Utilization 
            group by [Project ID]
        ) as util
        cross apply string_split(ProjectId, '.', 1)
)

, cte_project_code as (
    select ProjectId, Part as ProjectNameCode 
    from cte_project_id_parts
    where PartNum = 1
)

, cte_task_order as (
    select ProjectId, Part as TaskOrder 
    from cte_project_id_parts
    where PartNum = 2
)

, cte_option_year as (
    select ProjectId, Part as OptionYear 
    from cte_project_id_parts
    where PartNum = 3
)

, cte_clin as (
    select ProjectId, Part as CLIN 
    from cte_project_id_parts
    where PartNum = 4
)

, cte_sub_clin as (
    select ProjectId, Part as SubCLIN 
    from cte_project_id_parts
    where PartNum = 5
)

select * from cte_sub_clin

(80 rows affected)

Total execution time: 00:00:00.136

ProjectId SubCLIN BNPADM.CIV.COM.000.EXE EXE BNPADM.CIV.COM.000.PMO PMO BNPADM.CIV.COM.000.PRP PRP BNPADM.CIV.COM.000.SOL SOL BNPADM.CIV.DOS.000.PMO PMO BNPADM.CIV.DOS.000.PRP PRP BNPADM.CIV.DOS.000.SOL SOL BNPADM.CIV.DOT.000.PRP PRP BNPADM.CIV.DOT.000.SOL SOL BNPADM.CIV.EDU.000.EXE EXE BNPADM.CIV.EDU.000.FIN FIN BNPADM.CIV.EDU.000.PRP PRP BNPADM.CIV.EDU.000.SOL SOL BNPADM.CIV.HHS.000.EXE EXE BNPADM.CIV.HHS.000.FCV FCV BNPADM.CIV.HHS.000.FHL FHL BNPADM.CIV.HHS.000.FIN FIN BNPADM.CIV.HHS.000.PMO PMO BNPADM.CIV.HHS.000.PRP PRP BNPADM.CIV.HHS.000.SOL SOL BNPADM.CIV.NSF.000.CIV CIV BNPADM.CIV.NSF.000.EXE EXE BNPADM.CIV.NSF.000.FIN FIN BNPADM.CIV.NSF.000.PRP PRP BNPADM.CIV.NSF.000.SOL SOL BNPADM.CIV.OTH.000.CIV CIV BNPADM.CIV.OTH.000.EXE EXE BNPADM.CIV.OTH.000.FCV FCV BNPADM.CIV.OTH.000.FHL FHL BNPADM.CIV.OTH.000.FIN FIN BNPADM.CIV.OTH.000.PMO PMO BNPADM.CIV.OTH.000.PRP PRP BNPADM.CIV.OTH.000.SOL SOL BNPADM.CIV.USD.000.EXE EXE BNPADM.CIV.USD.000.FIN FIN BNPADM.CIV.USD.000.PRP PRP BNPADM.CIV.USD.000.SOL SOL BNPADM.DEF.ARM.000.EXE EXE BNPADM.DEF.ARM.000.FIN FIN BNPADM.DEF.ARM.000.PRP PRP BNPADM.DEF.ARM.000.SOL SOL BNPADM.DEF.NAV.000.EXE EXE BNPADM.DEF.NAV.000.FIN FIN BNPADM.DEF.NAV.000.PMO PMO BNPADM.DEF.NAV.000.PRP PRP BNPADM.DEF.OTH.000.PMO PMO BNPADM.DEF.OTH.000.PRP PRP BNPADM.DEF.OTH.000.SOL SOL BNPADM.HML.DHS.000.EXE EXE BNPADM.HML.DHS.000.FCV FCV BNPADM.HML.DHS.000.FHL FHL BNPADM.HML.DHS.000.FIN FIN BNPADM.HML.DHS.000.PMO PMO BNPADM.HML.DHS.000.PRP PRP BNPADM.HML.DHS.000.SOL SOL BNPADM.HML.OTH.000.PRP PRP NIH002.0092.00.0001.01 01 NRC001.0011.00.01.0001 0001 NRC001.0011.00.01.0002 0002 NRC001.0011.00.01.0004 0004 NRC001.0011.00.01.0009 0009 NRC001.0011.00.02.008A 008A NRC001.0011.00.02.008B 008B NRC001.0011.00.02.008C 008C NRC001.0031.00.01.0004 0004 NRC001.0031.00.02.009B 009B NRC001.0057.00.01.0001 0001 NRC001.0057.00.01.0002 0002 NRC001.0057.00.01.0003 0003 NRC001.0057.00.01.0009 0009 NRC001.0057.00.02.008A 008A NRC001.0057.00.02.008B 008B NRC001.0075.00.01.0004 0004 NRC001.0075.00.01.0009 0009 NRC001.0075.00.02.008A 008A NRC001.0075.00.02.008B 008B VLNT01.0002.02.1012.A A VLNT01.0002.02.1012.B B VLNT01.0002.03.3003.A A VLNT01.0002.03.3003.B B